In [ ]:
# This cell is for installing the COCO API, which helps read the annotation files.
# It's typically pre-installed on Kaggle, but this ensures it's available.
!pip install pycocotools --quiet

import os
import json
import random
import numpy as np
from pycocotools.coco import COCO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm


print("Libraries imported successfully.")

In [ ]:
# Define the paths to the dataset
# This path is standard for this dataset on Kaggle
BASE_PATH = '/kaggle/input/coco-2017-dataset/coco2017/'
IMG_PATH = os.path.join(BASE_PATH, 'train2017')
ANN_PATH = os.path.join(BASE_PATH,r 'annotations', 'instances_train2017.json')

# Initialize the COCO API for instance annotations
print("Loading annotations... This may take a moment.")
coco = COCO(ANN_PATH)
print("Annotations loaded.")

In [ ]:
# Get all image IDs from the training set
all_img_ids = coco.getImgIds()
print(f"Total images in the training set: {len(all_img_ids)}")

# Collect the first 20,000 image IDs
image_ids_20k = all_img_ids[:20000]

print(f"Collected {len(image_ids_20k)} image IDs to work with.")

In [ ]:
# Select 50 images from our 20k list for visualization
# We'll take the first 50
images_to_visualize = image_ids_20k[:50]

print(f"Starting visualization for {len(images_to_visualize)} images...")

for img_id in images_to_visualize:
    # 1. Load Image Info
    img_info = coco.loadImgs([img_id])[0]
    img_path = os.path.join(IMG_PATH, img_info['file_name'])
    
    # 2. Load Original Image (as RGB)
    original_image = Image.open(img_path).convert('RGB')
    
    # 3. Get all annotations for this image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    annotations = coco.loadAnns(ann_ids)
    
    # 4. Create the segmentation mask using numpy and coco.annToMask
    # This is more robust as it handles both polygon and RLE formats.
    
    # Create a blank numpy array for the combined mask
    combined_mask_np = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    
    # Iterate over all annotations (all subjects) for this image
    for ann in annotations:
        # coco.annToMask returns a binary 0/1 numpy array
        # This function correctly handles both RLE and polygon formats
        single_ann_mask_np = coco.annToMask(ann)
        
        # Combine masks using logical OR
        # This adds the current object's mask to the combined mask
        combined_mask_np = np.logical_or(combined_mask_np, single_ann_mask_np).astype(np.uint8)

    # Convert the final combined numpy mask (0/1) to a PIL Image (0/255)
    # We multiply by 255 to make the mask visible (white) for PIL
    mask_pil = Image.fromarray(combined_mask_np * 255, 'L')
    
    # 5. Create the final "Subject Isolated" image
    # This creates a black RGB background
    black_bg = Image.new('RGB', original_image.size, (0, 0, 0))
    
    # Use the PIL mask to composite the images
    # Where mask_pil is 255 (white), pixels from original_image are used.
    # Where mask_pil is 0 (black), pixels from black_bg are used.
    masked_image = Image.composite(original_image, black_bg, mask_pil)
    
    # 6. Visualize side-by-side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # Original Image
    ax1.imshow(original_image)
    ax1.set_title(f"Original Image: {img_info['file_name']}")
    ax1.axis('off')
    
    # Masked Image (Subject Isolated)
    ax2.imshow(masked_image)
    ax2.set_title("Masked Image (Subject Isolated)")
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

print("Visualization complete.")

In [ ]:
def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size)


In [ ]:
def normalize_image(image):
    return image / 255.0


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
# Cell 1: imports & params
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
# Params - change as needed
train_path = "/kaggle/input/coco-2017-dataset/coco2017/train2017"
val_path   = "/kaggle/input/coco-2017-dataset/coco2017/val2017"  # optional
use_val = False           # set True to include val images
max_images = 20000        # how many source images to process (limit)
save_sample_count = 20    # how many processed images to save for inspection
output_image_folder = "/kaggle/working/processed_images"
output_npy_path = "/kaggle/working/processed_images_all.npy"  # optional: set None to skip
os.makedirs(output_image_folder, exist_ok=True)
seed = 42


In [ ]:
# Cell 2: preprocessing helpers
def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size, interpolation=cv2.INTER_AREA)

def normalize_image(image):
    # convert to float32 and scale to [0,1]
    return image.astype(np.float32) / 255.0

# Keras ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.08,
    height_shift_range=0.08,
    shear_range=0.05,
    zoom_range=0.08,
    horizontal_flip=True,
    fill_mode='nearest'
)
np.random.seed(seed)


In [ ]:
# Cell 3: gather filenames
def get_image_list(train_path, use_val=False, val_path=None, max_images=20000):
    files = sorted(glob.glob(os.path.join(train_path, "*.jpg")) + glob.glob(os.path.join(train_path, "*.png")))
    if use_val and val_path:
        files += sorted(glob.glob(os.path.join(val_path, "*.jpg")) + glob.glob(os.path.join(val_path, "*.png")))
    # deduplicate and limit
    files = list(dict.fromkeys(files))[:max_images]
    return files

image_files = get_image_list(train_path, use_val=use_val, val_path=val_path, max_images=max_images)
print("Files to process:", len(image_files))
if len(image_files) == 0:
    print("No image files found — check paths. Example listing:", os.listdir(os.path.dirname(train_path))[:20])


In [ ]:
SAVE_PATH = "/kaggle/working/processed_images"
os.makedirs(SAVE_PATH, exist_ok=True)

print("Saving processed masked images...")

count = 0
for img_id in tqdm(image_ids_20k[:5000]):  # save first 5000
    img_info = coco.loadImgs([img_id])[0]
    img_path = os.path.join(IMG_PATH, img_info['file_name'])
    original_image = Image.open(img_path).convert('RGB')

    ann_ids = coco.getAnnIds(imgIds=img_id)
    annotations = coco.loadAnns(ann_ids)

    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    for ann in annotations:
        mask = np.logical_or(mask, coco.annToMask(ann)).astype(np.uint8)

    masked = Image.composite(original_image, Image.new('RGB', original_image.size, (0,0,0)), Image.fromarray(mask * 255, 'L'))
    masked = masked.resize((256, 256))
    
    save_name = f"masked_{img_id}.jpg"
    masked.save(os.path.join(SAVE_PATH, save_name))
    count += 1

print(f"Saved {count} processed masked images.")


In [ ]:
import os
import cv2
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

# Path to processed images (adjust if needed)
PROCESSED_PATH = "/kaggle/working/processed_images"

# Get list of processed image files
all_images = sorted(os.listdir(PROCESSED_PATH))
print(f"Total processed images found: {len(all_images)}")

# Select only 5000 for the pipeline
pipeline_images = all_images[:5000]
print(f"Using {len(pipeline_images)} images for pipeline building...")

# Example pipeline steps: resize, normalization, augmentation
def process_image(img_path):
    img = cv2.imread(os.path.join(PROCESSED_PATH, img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize to a smaller shape (adjust if needed)
    img_resized = cv2.resize(img, (256, 256))

    # Normalization
    img_norm = img_resized / 255.0

    # Simple Augmentation: horizontal flip (example)
    if random.random() > 0.5:
        img_aug = cv2.flip(img_norm, 1)
    else:
        img_aug = img_norm
    
    return img_aug

processed_samples = []

for img_file in tqdm(pipeline_images, desc="Building pipeline"):
    processed_samples.append(process_image(img_file))

print("✅ Pipeline processing complete.")

# ---- Display 50 random sample images ----
sample_images = random.sample(processed_samples, 50)
plt.figure(figsize=(15, 15))
for i, img in enumerate(sample_images):
    plt.subplot(5, 10, i+1)
    plt.imshow(img)
    plt.axis('off')
plt.suptitle("Sample 50 Augmented + Resized Images", fontsize=16)
plt.show()


In [ ]:
import os

print("Input root:", os.listdir("/kaggle/input"))

print("COCO dataset:", os.listdir("/kaggle/input/coco-2017-dataset"))


In [ ]:
import os

root = "/kaggle/input/coco-2017-dataset/coco2017"
print("Root contents:", os.listdir(root))

# Check for expected subfolders
for sub in ["train2017", "val2017", "annotations"]:
    path = os.path.join(root, sub)
    print(sub, "exists:", os.path.exists(path))


In [ ]:
import os
import cv2
from tqdm import tqdm

INPUT_DIR = "/kaggle/working/processed_images"      # RGB augmented images
OUTPUT_DIR = "/kaggle/working/processed_images_gray"  # Grayscale output folder

os.makedirs(OUTPUT_DIR, exist_ok=True)

image_files = sorted(os.listdir(INPUT_DIR))
print(f"Total RGB processed images found: {len(image_files)}")

for img_name in tqdm(image_files, desc="Converting to Grayscale"):
    img_path = os.path.join(INPUT_DIR, img_name)
    
    # Load image
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    # Convert BGR → Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Optional: resize if needed
    # gray = cv2.resize(gray, (224, 224))

    # Save grayscale image
    save_name = os.path.splitext(img_name)[0] + "_gray.jpg"
    cv2.imwrite(os.path.join(OUTPUT_DIR, save_name), gray)

print("✅ Grayscale conversion completed!")
print(f"Grayscale images saved to: {OUTPUT_DIR}")


In [ ]:
import matplotlib.pyplot as plt
import random

sample_files = random.sample(image_files, min(20, len(image_files)))

plt.figure(figsize=(12, 8))
for i, name in enumerate(sample_files):
    img = cv2.imread(os.path.join(OUTPUT_DIR, name.replace(".jpg", "_gray.jpg")))
    plt.subplot(4, 5, i+1)
    plt.imshow(img, cmap="gray")
    plt.axis("off")

plt.suptitle("Random Grayscale Converted Images", fontsize=16)
plt.show()


In [ ]:
import os
import numpy as np
from pycocotools.coco import COCO
from PIL import Image
from tqdm import tqdm
import cv2

# Input masked image path
IMG_DIR = "/kaggle/working/processed_images"

# Output mask path
MASK_DIR = "/kaggle/working/processed_masks"
os.makedirs(MASK_DIR, exist_ok=True)

# COCO path
BASE_PATH = '/kaggle/input/coco-2017-dataset/coco2017/'
ANN_PATH = os.path.join(BASE_PATH, 'annotations', 'instances_train2017.json')
coco = COCO(ANN_PATH)

# Extract image IDs from filenames
files = sorted(os.listdir(IMG_DIR))

print("Saving binary masks...")

for file in tqdm(files):
    if not file.startswith("masked_") or not file.endswith(".jpg"):
        continue

    img_id = int(file.replace("masked_", "").replace(".jpg", ""))  # recover COCO image id
    img_info = coco.loadImgs([img_id])[0]

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    for ann in anns:
        mask = np.logical_or(mask, coco.annToMask(ann)).astype(np.uint8)

    # Resize mask to 256x256 same as masked images
    mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)
    mask = (mask > 0).astype("uint8")

    np.save(os.path.join(MASK_DIR, file.replace(".jpg", ".npy")), mask)

print("✅ Mask creation complete!")
print("Masks saved in:", MASK_DIR)


In [ ]:
print("Images:", len(os.listdir("/kaggle/working/processed_images")))
print("Masks :", len(os.listdir("/kaggle/working/processed_masks")))


In [ ]:
IMG_DIR = "/kaggle/working/processed_images"
MASK_DIR = "/kaggle/working/processed_masks"
IMG_SIZE = (256, 256)


In [ ]:
import numpy as np
import os
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split

files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(".jpg")])

train_files, val_files = train_test_split(files, test_size=0.1, random_state=42)

class CocoDataGen(Sequence):
    def __init__(self, file_list, batch_size=8, shuffle=True):
        self.file_list = file_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.file_list) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.file_list)

    def __getitem__(self, idx):
        batch_files = self.file_list[idx*self.batch_size:(idx+1)*self.batch_size]
        X, Y = [], []
        for f in batch_files:
            img = cv2.imread(os.path.join(IMG_DIR, f), cv2.IMREAD_GRAYSCALE)
            img = img.astype("float32") / 255.0
            img = np.expand_dims(img, -1)

            mask = np.load(os.path.join(MASK_DIR, f.replace(".jpg", ".npy")))
            mask = np.expand_dims(mask.astype("float32"), -1)

            X.append(img)
            Y.append(mask)

        return np.array(X), np.array(Y)

train_gen = CocoDataGen(train_files, batch_size=8)
val_gen   = CocoDataGen(val_files,   batch_size=8, shuffle=False)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def build_unet(input_shape=(256, 256, 1)):
    inputs = Input(input_shape)

    def conv_block(x, filters):
        x = Conv2D(filters, 3, activation="relu", padding="same")(x)
        x = Conv2D(filters, 3, activation="relu", padding="same")(x)
        return x

    c1 = conv_block(inputs, 16); p1 = MaxPooling2D()(c1)
    c2 = conv_block(p1, 32); p2 = MaxPooling2D()(c2)
    c3 = conv_block(p2, 64); p3 = MaxPooling2D()(c3)
    c4 = conv_block(p3, 128); p4 = MaxPooling2D()(c4)

    bn = conv_block(p4, 256)

    u4 = UpSampling2D()(bn); u4 = Concatenate()([u4, c4]); c5 = conv_block(u4, 128)
    u3 = UpSampling2D()(c5); u3 = Concatenate()([u3, c3]); c6 = conv_block(u3, 64)
    u2 = UpSampling2D()(c6); u2 = Concatenate()([u2, c2]); c7 = conv_block(u2, 32)
    u1 = UpSampling2D()(c7); u1 = Concatenate()([u1, c1]); c8 = conv_block(u1, 16)

    outputs = Conv2D(1, 1, activation="sigmoid")(c8)

    return Model(inputs, outputs)

model = build_unet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    ]
)

model.save("/kaggle/working/unet_coco_allclasses.h5")
print("🎉 Model training complete & saved!")


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import os

# ---------------------------
# 1️⃣ Load the saved model
# ---------------------------
model_path = "/kaggle/working/unet_coco_allclasses.h5"
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully!")

# ---------------------------
# 2️⃣ Select random samples from validation set
# ---------------------------
num_samples = 3  # how many predictions to visualize
sample_files = random.sample(val_files, num_samples)

# ---------------------------
# 3️⃣ Helper function to load image & mask
# ---------------------------
def load_image_mask(img_filename):
    img_path = os.path.join(IMG_DIR, img_filename)
    mask_path = os.path.join(MASK_DIR, img_filename.replace(".jpg", ".png"))  # adjust extension if needed

    # Load image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0

    # Load mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask_resized = cv2.resize(mask, (IMG_SIZE, IMG_SIZE)) / 255.0
    mask_resized = np.expand_dims(mask_resized, axis=-1)

    return img, img_resized, mask, mask_resized

# ---------------------------
# 4️⃣ Predict & Plot results
# ---------------------------
for img_file in sample_files:
    orig_img, img_input, orig_mask, mask_input = load_image_mask(img_file)

    pred_mask = model.predict(np.expand_dims(img_input, axis=0))[0]
    pred_mask = (pred_mask > 0.5).astype(np.uint8)  # thresholding

    # Plotting
    plt.figure(figsize=(12,4))
    
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(orig_img)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(orig_mask, cmap="gray")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask")
    plt.imshow(pred_mask.squeeze(), cmap="gray")
    plt.axis("off")

    plt.show()
